# **Web Scrapping with Python**

The objective of this project is to scrap Mobile Legends data from this url "https://zathong.com/mobile-legends-builds/" and perform some simple EDA on that data.

## Import Libraries :
- pandas : 
- BeautifulSoup
- requests : 

In [20]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

## Make HTTPS requests

In this example we get "Response [200]" which indicates that the requests has succeeded.

In [27]:
url = "https://zathong.com/mobile-legends-builds/"
response = requests.get(url)
print(response)

<Response [200]>


## Parse HTML Content
Create a Beautiful Soup object by passing the HTML content to the Beautiful Soup constructor. This will allow you to navigate and search through the HTML structure.

In [36]:
content = response.text
soup = BeautifulSoup(content, 'html.parser')

## Find and Extract Data

In [37]:
## Function to get list of hero's
def get_li_character(soup):
        
        soup = soup
        
        li = soup.find(class_='is-layout-flex wp-container-6 wp-block-columns has-background').find_all('a')
        li = list(set([item['href'] for item in li]))
        
        return li

## Get Link for each Hero
li_hero = get_li_character(soup)[:3]

## Function to get hero data

In [41]:
def testing(li_hero):
## Create Empty List
    data = []
    for hero in li_hero :
        hero_data = []
                
        response = requests.get(hero)
        content = response.text
        soup = BeautifulSoup(content, 'html.parser')
        information = soup.find(class_='entry-content')
                
        try :
            ## Hero Name
            name = information.find('figure').text
            
            ## Hero Role
            role = information.find_all('p', class_='has-text-align-center')[0].text.split(':')[1].split(' ')[0]
            speciality = information.find_all('p', class_='has-text-align-center')[1].text.split(':')[1]
            lane = information.find_all('p', class_='has-text-align-center')[2].text.split(':')[1]
            tier = information.find_all('p', class_='has-text-align-center')[3].text.split(':')[1]

            ## Basic Stat
            mov_speed = information.find_all('td')[:10][0].text.split(':')[1]
            physical_att = information.find_all('td')[:10][1].text.split(':\xa0')[1]
            magic_pwr = information.find_all('td')[:10][2].text.split(':\xa0')[1]
            physical_def = information.find_all('td')[:10][3].text.split(':')[1]
            magic_def = information.find_all('td')[:10][4].text.split(':\xa0')[1]
            hp = information.find_all('td')[:10][5].text.split(':')[1]
            mana = information.find_all('td')[:10][6].text.split(':')[1]
            att_speed = information.find_all('td')[:10][7].text.split(':')[1]
            hp_reg = information.find_all('td')[:10][8].text.split(':')[1]
            mana_reg = information.find_all('td')[:10][9].text.split(':')[1]
        except :
            pass
        
        basic_info = [name, role, speciality, lane, tier]
        basic_stat = [mov_speed, physical_att, magic_pwr, physical_def, magic_def, hp, mana, att_speed, hp_reg, mana_reg]
        
        data.append(basic_info+basic_stat)
    return data

In [43]:
data = testing(li_hero)

## Store Data

In [47]:
## Function to Create Dataframe and Store Data into Dataframe. 
def store_data(data):
    col_names = ['name','role','speciality','lane','tier','mov_speed','physical_att','magic_pwr','physical_def','magic_def','hp','mana','att_speed','hp_reg','mana_reg']
    df = pd.DataFrame(data, columns=col_names)

    df[df.columns[5:].to_list()] = df[df.columns[5:].to_list()].replace(' ','')

    df['mov_speed'] = pd.to_numeric(df.mov_speed, downcast='integer', errors='coerce')
    df['physical_att'] = pd.to_numeric(df.physical_att, downcast='integer', errors='coerce')
    df['magic_pwr'] = pd.to_numeric(df.magic_pwr, downcast='integer', errors='coerce')
    df['physical_def'] = pd.to_numeric(df.physical_def, downcast='integer', errors='coerce')
    df['magic_def'] = pd.to_numeric(df.magic_def, downcast='integer', errors='coerce')
    df['hp'] = pd.to_numeric(df.hp, downcast='integer', errors='coerce')
    df['mana'] = pd.to_numeric(df.mana, downcast='integer', errors='coerce')
    df['att_speed'] = pd.to_numeric(df.att_speed, downcast='float', errors='coerce')
    df['hp_reg'] = pd.to_numeric(df.hp_reg, downcast='float', errors='coerce')
    df['mana_reg'] =  pd.to_numeric(df.mana_reg, downcast='float', errors='coerce')

    return df

In [48]:
store_data(data)

,name,role,speciality,lane,tier,mov_speed,physical_att,magic_pwr,physical_def,magic_def,hp,mana,att_speed,hp_reg,mana_reg
0,Zilong,,Chase/Charge,"EXP, Jungle",A+,265,123,0,25,NaN,2689,405,0.964,7.0,3.2
1,Layla,,Burst/Control,Gold,A+,240,125,0,15,NaN,2500,424,0.850,5.4,2.8
2,Johnson,,Support | Crowd Control,Roam,S,255,120,0,27,10.0,2809,0,0.826,8.4,2.4
